### Notebook: Normalize Expression Data (`normalize_expr_mat.ipynb`)

This notebook is dedicated to normalizing new expression data using the Binning-By-Gene method, tailored for use with the GeneRAIN models. It includes instructions and code snippets for processing your data in line with the requirements of different GeneRAIN models.


In [1]:
import anndata
import numpy as np
import pandas as pd
import pickle
from data.adata import Adata
from utils.config_loader import Config
import os
from data.GetBinsByGeneForNewSamples import get_bins_by_gene_for_new_samples

- **Selecting Binning Boundaries**: 
  - Specify the files to obtain binning boundaries for Binning-By-Gene normalization.
  - Note: The gene set in the `coding_lncrna` model differs from other models.
  - Set `gene_emb_name` to `'coding_lncrna'` for the `GeneRAIN.GPT_protein_coding_lncRNAs` model.
  - Set `gene_emb_name` to `'gene2vec'` for the `GeneRAIN.GPT_Binning_By_Gene`, `GeneRAIN.BERT_Pred_Genes_Binning_By_Gene`, or `GeneRAIN.BERT_Pred_Expr_Binning_By_Gene` models.


In [2]:
config = Config()
# Set gene_emb_name to 'coding_lncrna' if you are preparing dataset for GeneRAIN.GPT_protein_coding_lncRNAs model
# Set gene_emb_name to 'gene2vec' if you are preparing dataset for GeneRAIN.GPT_Binning_By_Gene, 
# GeneRAIN.BERT_Pred_Genes_Binning_By_Gene or GeneRAIN.BERT_Pred_Expr_Binning_By_Gene models, as they are using gene set overlaping with gene2vec
gene_emb_name = "coding_lncrna"
if gene_emb_name == "coding_lncrna":
    samples_by_genes_subsampled_file = config.proj_path + "/data/external/ARCHS/normalize_each_gene/human_gene_v2.2_with_zero_expr_genes_bin_tot2000_coding_lncrna_0.005_subsampled.npy"
    subsampled_gene_symbol_file = config.proj_path + "/data/external/ARCHS/normalize_each_gene/human_gene_v2.2_with_zero_expr_genes_bin_tot2000_coding_lncrna_0.005_subsampled.gene_symbols.txt"
elif gene_emb_name == "gene2vec":
    samples_by_genes_subsampled_file = config.proj_path + "/data/external/ARCHS/normalize_each_gene/human_gene_v2.2_with_zero_expr_genes_bin_tot2000_gene2vec_0.005_subsampled.npy"
    subsampled_gene_symbol_file = config.proj_path + "/data/external/ARCHS/normalize_each_gene/human_gene_v2.2_with_zero_expr_genes_bin_tot2000_gene2vec_0.005_subsampled.gene_symbols.txt"    

- **Processing Expression Matrix**:
  - Obtain the expression matrix and the corresponding gene symbols to prepare the dataset for model input.

In [7]:
gene_expr_data_file = config.proj_path + "/data/examples/ReplogleWeissman2022_K562_essential.h5ad.mean_agg.pickle"
with open(gene_expr_data_file, 'rb') as f:
    adata = pickle.load(f)
# the list of sample or cell names of the matrix
cell_group_names = adata.obs_names
# the list of gene symbols of the matrix
gene_names = adata.var_names
# the expression matrix
sample_by_gene_expr_matrix = adata.X

- **Applying Binning-By-Gene Normalization**:
  - Utilize the `get_bins_by_gene_for_new_samples` function to normalize the expression matrix. The function's documentation provides further details.

    Parameters:
    - samples_by_genes_subsampled_file (str): File containing the subsampled genes by samples matrix.
    - subsampled_gene_symbol_file (str): File containing gene symbols corresponding to the rows of the subsampled matrix.
    - gene_by_sample_expr_mat (numpy.ndarray): Gene by sample expression matrix to be processed and binned.
    - quantified_gene_list (list or numpy.ndarray): List of gene symbols corresponding to the rows of gene_by_sample_expr_mat.
    - output_prefix (str, optional): Prefix for output files such as statistics dataframe. None for no output.
    - num_gene_bin (int, optional): The number of bins to be used for discretization. Defaults to 2000.
    - min_total_count (int, optional): Minimum total count to consider a sample as valid. Defaults to -1.

    Returns:
    - gene_by_sample_mat_binned (numpy.ndarray): Binned version of the input gene_by_sample_expr_mat.
    - bool_for_if_samples_included_in_returned_mat (numpy.ndarray): Boolean array indicating if the samples in the input matrix are included in the processed matrix.
    - gene_exist_in_subsampled (numpy.ndarray): Boolean array indicating which genes from the input matrix are present in the subsampled matrix.
    - stats_df (pandas.DataFrame): DataFrame containing statistics for each sample in the input matrix.

    Notes:
    The function uses a subsampled gene expression dataset to bin the input gene expression dataset. The binned values correspond to the discretized expression values of the genes, based on the subsampled data.


In [5]:

gene_by_sample_mat_binned, bool_for_if_samples_included_in_returned_mat, gene_exist_in_subsampled, stats_df = get_bins_by_gene_for_new_samples(samples_by_genes_subsampled_file, 
                                                                                                                                                 subsampled_gene_symbol_file, 
                                                                                                                                                 gene_by_sample_expr_mat=sample_by_gene_expr_matrix.T, 
                                                                                                                                                 quantified_gene_list=gene_names, 
                                                                                                                                                 output_prefix=None, 
                                                                                                                                                 num_gene_bin= 2000,  
                                                                                                                                                 min_total_count=-1)



genes_by_samples_subsampled_mat shape (31769, 2054)
processing samples from 0 to 4
processing samples from 10 to 14
processing samples from 20 to 24
processing samples from 30 to 34
processing samples from 40 to 44
processing samples from 50 to 54
processing samples from 60 to 64
processing samples from 70 to 74
processing samples from 80 to 84
processing samples from 90 to 94
processing samples from 100 to 104
processing samples from 110 to 114
processing samples from 120 to 124
processing samples from 130 to 134
processing samples from 140 to 144
processing samples from 150 to 154
processing samples from 160 to 164
processing samples from 170 to 174
processing samples from 180 to 184
processing samples from 190 to 194
processing samples from 200 to 204
processing samples from 210 to 214
processing samples from 220 to 224
processing samples from 230 to 234
processing samples from 240 to 244
processing samples from 250 to 254
processing samples from 260 to 264
processing samples from 2

- **Save the normalized output**
    - Save the normalized output as an Adata object and export it in both `.pkl` and `.tsv` formats for subsequent use.


In [6]:
binned_adata = Adata(np.array(cell_group_names)[bool_for_if_samples_included_in_returned_mat], 
                     np.array(gene_names)[gene_exist_in_subsampled],
                     gene_by_sample_mat_binned.T[:,gene_exist_in_subsampled]
                    )

output_pickle_file = config.proj_path + f"/data/examples/ReplogleWeissman2022_K562_essential.h5ad.mean_agg.{gene_emb_name}.binned.pkl"
with open(output_pickle_file, 'wb') as f:
    pickle.dump(binned_adata, f)
print(f"expr data saved to {output_pickle_file}.")

output_tsv_file = config.proj_path + f"/data/examples/ReplogleWeissman2022_K562_essential.h5ad.mean_agg.{gene_emb_name}.binned.tsv"
binned_adata.save_matrix_to_tsv(output_tsv_file)
print(f'Data saved to {output_tsv_file}.')

expr data saved to /g/data/yr31/zs2131/tasks/2023/RNA_expr_net/DeepGeneNet//data/examples/ReplogleWeissman2022_K562_essential.h5ad.mean_agg.coding_lncrna.binned.pkl!
Data saved to /g/data/yr31/zs2131/tasks/2023/RNA_expr_net/DeepGeneNet//data/examples/ReplogleWeissman2022_K562_essential.h5ad.mean_agg.coding_lncrna.binned.tsv
